In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import warnings
warnings.filterwarnings('ignore' )
from bs4 import BeautifulSoup
from io import BytesIO
import requests
import json
from PIL import ImageFile,Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

img_size = (224, 224)
batch_size = 32

def create_model():
    dataset_dir = "D:/imagemod"

    train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        os.path.join(dataset_dir, 'train'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        os.path.join(dataset_dir, 'val1'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary'
    )
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    model = Sequential([
         base_model,
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )

    evaluation_result = model.evaluate(validation_generator)
    
    evaluation_dict = {
        "validation_loss": evaluation_result[0],
        "validation_accuracy": evaluation_result[1]
    }

    json_data = json.dumps(evaluation_dict)
    print("Validation Accuracy:", evaluation_result[1])
    return model

create_model()

Found 6253 images belonging to 2 classes.
Found 6221 images belonging to 2 classes.
195/195 [==============================] - 435s 2s/step - loss: 0.1797 - accuracy: 0.9396
Validation Accuracy: 0.9395595788955688


In [21]:
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import load_model
from PIL import Image
import numpy as np

def input_model_image(img_path):
    train_model = False

    if train_model:
        model = create_model()
    else:
        model_path = "D:/image_moderation_model.h5"
        model = tf.keras.models.load_model(model_path)

    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    prediction = model.predict(img_array)[0][0]
    return prediction

img_path = 'C:/Users/i310thgeN/Documents/webpage/djangoproject/blog/static/images/fashion.jpg'
prediction = input_model_image(img_path)

if prediction > 0.5:
    print('Image not approved')
else:
    print('Image Approved')
     

Image Approved


In [13]:
import numpy as np
import pandas as pd
import pickle

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [6]:


data=pd.read_csv('D:/labeled_data.csv')
#print(data)
data = data.dropna()
#X = ['hi', 'idiot', 'u are black']
#y = [2, 0, 1]
X=data['tweet']
y=data['class']
#print('X',X)
#print(y)
def create_and_train_model(X, y):
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(sequences)

    
    X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

    
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_padded.shape[1]))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=len(np.unique(y_encoded)), activation='softmax'))  

    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

    
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")

    model.save('D:/content_moderation_model.h5')
    print("Model saved.")

    
    with open('D:/tokenizer.pkl', 'wb') as file:
        pickle.dump(tokenizer, file)

    with open('D:/ label_encoder.pkl', 'wb') as file:
        pickle.dump(label_encoder, file)

    print("Tokenizer and LabelEncoder saved.")

    

    return model, tokenizer, label_encoder

create_and_train_model(X,y)

Epoch 1/5
496/496 [==============================] - 119s 217ms/step - loss: 0.4546 - accuracy: 0.8499 - val_loss: 0.2896 - val_accuracy: 0.9070
Epoch 2/5
496/496 [==============================] - 104s 210ms/step - loss: 0.2287 - accuracy: 0.9225 - val_loss: 0.3169 - val_accuracy: 0.8954
Epoch 3/5
496/496 [==============================] - 104s 210ms/step - loss: 0.1253 - accuracy: 0.9573 - val_loss: 0.3736 - val_accuracy: 0.8860
Epoch 4/5
496/496 [==============================] - 111s 223ms/step - loss: 0.0625 - accuracy: 0.9800 - val_loss: 0.4974 - val_accuracy: 0.8858
Epoch 5/5
155/155 [==============================] - 7s 42ms/step - loss: 0.5801 - accuracy: 0.8794
Test Accuracy: 0.8793625235557556
Model saved.
Tokenizer and LabelEncoder saved.


(<keras.engine.sequential.Sequential at 0x2100417c970>,
 LabelEncoder())

In [22]:
import warnings
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
warnings.filterwarnings('ignore' )
import pickle
def predict_with_model(model, tokenizer, label_encoder, input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(input_sequence, maxlen=model.input_shape[1])

   
    predicted_probabilities = model.predict(padded_sequence)
    predicted_class = np.argmax(predicted_probabilities)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    confidence = predicted_probabilities[0][predicted_class]

    if confidence <= 0.6:
        predicted_label=2
    
    return predicted_label

def load_saved_model():
    
    loaded_model = load_model('D:/content_moderation_model.h5')

    
    with open('D:/tokenizer.pkl', 'rb') as file:
        loaded_tokenizer = pickle.load(file)

    with open('D:/ label_encoder.pkl', 'rb') as file:  # Corrected file name
        loaded_label_encoder = pickle.load(file)

    print("Model, Tokenizer, and LabelEncoder loaded.")
    return loaded_model, loaded_tokenizer, loaded_label_encoder


data=pd.read_csv('D:/labeled_data.csv')

X=data['tweet']
y=data['class']

def input_model(content):
   
    prediction=[]
    train_model=False
    if train_model:
        trained_model, tokenizer, label_encoder = create_and_train_model(X, y)
    else:
        trained_model, tokenizer, label_encoder = load_saved_model()

    result_string = content.replace('"', '').replace("'", '')

    soup = BeautifulSoup(content, 'html.parser')
    p_tag = soup.find_all('p')
    #print("p_tag",p_tag)
    for i in p_tag:
        #print(i)
        cleaned_text = i.get_text(separator=' ', strip=True) if i else ''
        #print(cleaned_text)
        a=predict_with_model(trained_model, tokenizer, label_encoder, cleaned_text)
        prediction.append(a)
        #print("Predictions",prediction)
    
    return prediction

a=input_model('<p>You are nothing but a waste of space in this bloody world... idiot i hate u so much you black people dont belong here hell dammit shit bloody losers</p>')
for i in a:
    if(i==0 or i==1):
        print("Inappropriate Content")
    else:
        print("Accepted Content")

Model, Tokenizer, and LabelEncoder loaded.
1/1 [==============================] - 1s 1s/step
Inappropriate Content
